In [171]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.naive_bayes import GaussianNB
#  importando los modulos confusion_matrix, roc_curve, auc, accuracy_score de sklearn.metrics
from sklearn.metrics import confusion_matrix, roc_curve, auc, roc_auc_score, accuracy_score, classification_report
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/datatn-bancolombia-2019/DT19_Datos_transaccionales_predict.csv
/kaggle/input/datatn-bancolombia-2019/DT19_Datos_transaccionales_train.csv
/kaggle/input/datatn-bancolombia-2019/Metadatos Datos_Var_Rpta.xlsx
/kaggle/input/datatn-bancolombia-2019/DT19_IDs_predict.csv
/kaggle/input/datatn-bancolombia-2019/Metadatos Datos_Trasaccionales.xlsx
/kaggle/input/datatn-bancolombia-2019/Trminos y Condiciones Datatn 2019.pdf
/kaggle/input/datatn-bancolombia-2019/presentacin Dataton BC 2019.pdf
/kaggle/input/datatn-bancolombia-2019/DT19_maestro_cdgtrn_cdgrpta.csv
/kaggle/input/datatn-bancolombia-2019/DT19_Datos_Var_Rpta_train_lite.csv
/kaggle/input/datatn-bancolombia-2019/DT19_Datos_transaccionales_train_lite.csv
/kaggle/input/datatn-bancolombia-2019/Metadatos maestro_cdgtrn_cdgrpta.xlsx
/kaggle/input/datatn-bancolombia-2019/DT19_Datos_Var_Rpta_train.csv


In [172]:
dataY = pd.read_csv('/kaggle/input/datatn-bancolombia-2019/DT19_Datos_Var_Rpta_train.csv')
dataX = pd.read_csv('/kaggle/input/datatn-bancolombia-2019/DT19_Datos_transaccionales_train.csv')
dataX['fecha_trxn'] = dataX['fecha_trxn'].astype('datetime64[ns]')

In [173]:
dataY.head()

,id,f_analisis,var_rpta,segmento
0,1,201803,0,4
1,2,201604,0,0
2,3,201608,0,5
3,4,201706,0,4
4,5,201703,0,4


## Define function to Group by client id the amount of transactions on each chanel and merge the different parts of the dataset

In [198]:
def getFeatures(dataX): 
    # Info by chanel
    grpcan = dataX.groupby(['id', 'canal'])['sesion'].nunique()
    trxcanal = grpcan.unstack(level=-1)
    trxcanal.columns = trxcanal.columns.values
    trxcanal = trxcanal.fillna(0)
    trxcanal['id'] = trxcanal.index.values
    del trxcanal.index.name
    
    grpdisp = dataX.groupby(['id', 'disposit'])['vlrtran'].aggregate('mean')
    trxcanal2 = grpdisp.unstack(level=-1)
    trxcanal2.columns = trxcanal2.columns.values
    trxcanal2 = trxcanal2.fillna(0)
    trxcanal2['id'] = trxcanal2.index.values
    del trxcanal2.index.name
    
    trxcanal3 = dataX.groupby(['id', 'sesion'])['fecha_trxn'].aggregate(['min', 'max'])
    trxcanal3.columns = trxcanal3.columns.values
    trxcanal3['diff'] = (trxcanal3['max'] - trxcanal3['min']) / np.timedelta64(1, 'm')
    trxcanal3 = trxcanal3.drop("min", axis=1)
    trxcanal3 = trxcanal3.drop("max", axis=1)
    trxcanal3 = trxcanal3.reset_index()
    trxcanal3 = trxcanal3.groupby(['id'])['diff'].aggregate(['sum'])
    trxcanal3['sum'] = np.log(trxcanal3['sum'] + 1)
    trxcanal3['id'] = trxcanal3.index.values
    del trxcanal3.index.name

    trxcanal = pd.merge(trxcanal, trxcanal2, left_on = 'id', right_on = 'id')

    return pd.merge(trxcanal, trxcanal3, left_on = 'id', right_on = 'id')

def mergeParts(dataA, dataB):
    data = pd.merge(dataA, dataB, left_on = 'id', right_on = 'id')
    #Convert segmento from categorical to dummy
    dummies = pd.get_dummies(data['segmento'], prefix = 'segmento')
    data = pd.concat([data.drop('segmento', axis = 1), dummies], axis = 1)
    return data

## Join the train info with the transactional info

In [175]:
data = mergeParts(dataY, getFeatures(dataX))
#data['sum'] = np.log(data['sum'] + 1)

In [176]:
data.head()

,id,f_analisis,var_rpta,CANAL_1,CANAL_2,CANAL_3,CANAL_4,CANAL_5,CANAL_6,CANAL_7,...,DISPOSIT_7,DISPOSIT_8,DISPOSIT_9,sum,segmento_0,segmento_1,segmento_2,segmento_3,segmento_4,segmento_5
0,1,201803,0,0.0,0.0,0.0,0.0,69.0,0.0,0.0,...,0.0,0.0,0.0,3.583007,0,0,0,0,1,0
1,2,201604,0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,1,0,0,0,0,0
2,3,201608,0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.609741,0,0,0,0,0,1
3,4,201706,0,0.0,0.0,0.0,0.0,618.0,1.0,0.0,...,0.0,0.0,0.0,12.282464,0,0,0,0,1,0
4,5,201703,0,0.0,1.0,0.0,0.0,70.0,46.0,0.0,...,0.0,0.0,0.0,16.062500,0,0,0,0,1,0


## Very simple experimental setup

In [178]:
colnames = data.columns.values.tolist()

predictors = colnames[3:len(colnames)]
targetName = 'var_rpta'

# Create a new array with the added features: features_two

features = data[predictors]
target = data[targetName]
# Split the data into train and test
trainX, testX, trainY, testY = train_test_split(features, target, test_size = 0.2, stratify = target )

print('Training: ')
print(trainX.shape, trainY.shape)
print(trainY.sum()/trainY.count())
print('Test: ')
print(testX.shape, testY.shape)
print(testY.sum()/testY.count())


Training: 
(61228, 28) (61228,)
0.07218919448618279
Test: 
(15308, 28) (15308,)
0.07218447870394565


## We need to undersample the dataset to avoid the unbalance of classes

In [179]:
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler()
X_res, y_res = rus.fit_resample(trainX, trainY)
#rus2 = RandomUnderSampler(random_state=231)
#X_res2, y_res2 = rus2.fit_resample(trainX, trainY)
#X_res = np.concatenate((X_res, X_res2))
#y_res = np.concatenate((y_res, y_res2))


## Train Neural network

In [195]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

# NN is sensitive to data scale. We must normilize
scaler = StandardScaler()  
trainXX = np.log(X_res.copy() + 1)
testXX =  np.log(testX.copy() + 1)



# Don't cheat - fit only on training data
scaler.fit(trainX)  
trainXX = scaler.transform(trainXX)  
# apply same transformation to test data
testXX = scaler.transform(testXX)  

model3 = MLPClassifier(solver='adam', shuffle=True, learning_rate_init= 0.0001, batch_size=300, alpha=1, activation = 'tanh', max_iter=5000, tol = 1e-14, hidden_layer_sizes = [60, 30, 10], learning_rate='adaptive', verbose = True)
model3.fit(trainXX, y_res)                     

#Print the score on the train data
print("On training")
print(model3.score(trainXX, y_res))
print(confusion_matrix(model3.predict(trainXX), y_res))

#Print the score on the test data
print("\nOn test")
print(model3.score(testXX, testY))
print(confusion_matrix(model3.predict(testXX), testY))

print("ROC discrete: ")
print(roc_auc_score(y_res, model3.predict(trainXX)))
print(roc_auc_score(testY,model3.predict(testXX)))

print("ROC prob: ")
print(roc_auc_score(y_res, [row[1] for row in model3.predict_proba(trainXX)]))
print(roc_auc_score(testY,[row[1] for row in model3.predict_proba(testXX)]))

Iteration 1, loss = 0.96394866
Iteration 2, loss = 0.92596365
Iteration 3, loss = 0.89430714
Iteration 4, loss = 0.86797295
Iteration 5, loss = 0.84686742
Iteration 6, loss = 0.83015931
Iteration 7, loss = 0.81715593
Iteration 8, loss = 0.80708318
Iteration 9, loss = 0.79919559
Iteration 10, loss = 0.79277323
Iteration 11, loss = 0.78748539
Iteration 12, loss = 0.78312450
Iteration 13, loss = 0.77924667
Iteration 14, loss = 0.77578308
Iteration 15, loss = 0.77272575
Iteration 16, loss = 0.76992311
Iteration 17, loss = 0.76741023
Iteration 18, loss = 0.76505544
Iteration 19, loss = 0.76270499
Iteration 20, loss = 0.76056629
Iteration 21, loss = 0.75845068
Iteration 22, loss = 0.75653787
Iteration 23, loss = 0.75459405
Iteration 24, loss = 0.75278947
Iteration 25, loss = 0.75093535
Iteration 26, loss = 0.74922999
Iteration 27, loss = 0.74752844
Iteration 28, loss = 0.74586621
Iteration 29, loss = 0.74417042
Iteration 30, loss = 0.74265673
Iteration 31, loss = 0.74103298
Iteration 32, los

Iteration 253, loss = 0.64262286
Iteration 254, loss = 0.64264167
Iteration 255, loss = 0.64256212
Iteration 256, loss = 0.64238470
Iteration 257, loss = 0.64228824
Iteration 258, loss = 0.64233344
Iteration 259, loss = 0.64209733
Iteration 260, loss = 0.64206766
Iteration 261, loss = 0.64193742
Iteration 262, loss = 0.64174005
Iteration 263, loss = 0.64171403
Iteration 264, loss = 0.64152332
Iteration 265, loss = 0.64146090
Iteration 266, loss = 0.64140061
Iteration 267, loss = 0.64129973
Iteration 268, loss = 0.64119481
Iteration 269, loss = 0.64123633
Iteration 270, loss = 0.64111475
Iteration 271, loss = 0.64099957
Iteration 272, loss = 0.64083696
Iteration 273, loss = 0.64088225
Iteration 274, loss = 0.64072241
Iteration 275, loss = 0.64060033
Iteration 276, loss = 0.64053419
Iteration 277, loss = 0.64043890
Iteration 278, loss = 0.64030790
Iteration 279, loss = 0.64036666
Iteration 280, loss = 0.64026234
Iteration 281, loss = 0.64011291
Iteration 282, loss = 0.64005716
Iteration 

Iteration 506, loss = 0.63061568
Iteration 507, loss = 0.63073624
Iteration 508, loss = 0.63063377
Iteration 509, loss = 0.63070623
Iteration 510, loss = 0.63064338
Iteration 511, loss = 0.63061749
Iteration 512, loss = 0.63056030
Iteration 513, loss = 0.63056989
Iteration 514, loss = 0.63051967
Iteration 515, loss = 0.63042560
Iteration 516, loss = 0.63042392
Iteration 517, loss = 0.63062414
Iteration 518, loss = 0.63046169
Iteration 519, loss = 0.63052373
Iteration 520, loss = 0.63045926
Iteration 521, loss = 0.63039847
Iteration 522, loss = 0.63034206
Iteration 523, loss = 0.63033949
Iteration 524, loss = 0.63047829
Iteration 525, loss = 0.63037546
Iteration 526, loss = 0.63036449
Iteration 527, loss = 0.63030105
Iteration 528, loss = 0.63015536
Iteration 529, loss = 0.63016681
Iteration 530, loss = 0.63013192
Iteration 531, loss = 0.63020394
Iteration 532, loss = 0.63019958
Iteration 533, loss = 0.63020302
Iteration 534, loss = 0.63017575
Iteration 535, loss = 0.63004944
Iteration 

Iteration 758, loss = 0.62733925
Iteration 759, loss = 0.62749333
Iteration 760, loss = 0.62733000
Iteration 761, loss = 0.62731003
Iteration 762, loss = 0.62727939
Iteration 763, loss = 0.62725219
Iteration 764, loss = 0.62718926
Iteration 765, loss = 0.62739406
Iteration 766, loss = 0.62729464
Iteration 767, loss = 0.62717772
Iteration 768, loss = 0.62724113
Iteration 769, loss = 0.62726077
Iteration 770, loss = 0.62729498
Iteration 771, loss = 0.62718390
Iteration 772, loss = 0.62721171
Iteration 773, loss = 0.62723303
Iteration 774, loss = 0.62713005
Iteration 775, loss = 0.62707217
Iteration 776, loss = 0.62729614
Iteration 777, loss = 0.62712395
Iteration 778, loss = 0.62717216
Iteration 779, loss = 0.62712978
Iteration 780, loss = 0.62715465
Iteration 781, loss = 0.62715667
Iteration 782, loss = 0.62706541
Iteration 783, loss = 0.62715468
Iteration 784, loss = 0.62717437
Iteration 785, loss = 0.62702737
Iteration 786, loss = 0.62710952
Iteration 787, loss = 0.62711228
Iteration 

In [193]:
print("ROC: ")
print(roc_auc_score(y_res, [row[1] for row in model3.predict_proba(trainXX)]))
print(roc_auc_score(testY,[row[1] for row in model3.predict_proba(testXX)]))

ROC: 
0.7542969533793329
0.7050748630953311


In [191]:
[row[1] for row in model3.predict_proba(trainXX)]

[0.5988540628851996,
 0.5138412566484156,
 0.6023030019108206,
 0.3341681795637401,
 0.6772005053702912,
 0.6300100051178885,
 0.007492735193124078,
 0.40997718502773156,
 0.5576994337481822,
 0.20055122400383085,
 0.3522433010308402,
 0.6990984643067116,
 0.4794964141920466,
 0.3057526682356294,
 0.5397303406956447,
 0.20175335487961144,
 0.7647478583535936,
 0.22564565035333686,
 0.5376124591622246,
 0.8200012135170103,
 0.4628902294221251,
 0.18734124599541513,
 0.19291595619320148,
 0.21081878996069492,
 0.2658013212178857,
 0.6635869989938901,
 0.15510245948689655,
 0.5415942147375113,
 0.4761172512742415,
 0.17787103982009125,
 0.4765438551835415,
 0.43834291470998044,
 0.3188364958797625,
 0.6722486614188157,
 0.17366029728015148,
 0.462201341306472,
 0.40662187747944106,
 0.4269087139549434,
 0.6426028155247276,
 0.5359861624182807,
 0.4824926044195202,
 0.4862351735822714,
 0.3592179602126865,
 0.6497929452477097,
 0.17553825056754901,
 0.514248720763575,
 0.4441767808056016,


## Train a Naive Bayes model

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()

trainX = X_res.copy()
testXX = testX.copy()

model2 = gnb.fit(trainX, y_res)

#Print the score on the train data
print("On training")
print(model2.score(trainX, y_res))
print(confusion_matrix(model2.predict(trainX), y_res))

#Print the score on the test data
print("\nOn test")
print(model2.score(testX, testY))
print(confusion_matrix(model2.predict(testX), testY))

print("ROC: ")
print(roc_auc_score(y_res, model3.predict(trainXX)))
print(roc_auc_score(testY, model3.predict(testXX)))

## Make a new prediction and save it in the output file

In [199]:
data2Y = pd.read_csv('/kaggle/input/datatn-bancolombia-2019/DT19_IDs_predict.csv')
data2X = pd.read_csv('/kaggle/input/datatn-bancolombia-2019/DT19_Datos_transaccionales_predict.csv')

data2X['fecha_trxn'] = dataX['fecha_trxn'].astype('datetime64[ns]')

data2 = mergeParts(data2Y, getFeatures(data2X))

features = data2[predictors]

test2XX = scaler.transform(features)  
#Make a prediction
data2Y['probabilidad'] = [row[1] for row in model3.predict_proba(test2XX)] #model3.predict(test2XX)

data_final = data2Y[['id', 'probabilidad']]

# Save to a file. Find it in /kaggle/output/working/
data_final.to_csv("DT19_IDs_predict.csv", index=False)
